In [ ]:
!pip install geemap retry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import numpy as np
import io
import requests

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-photobackupisf')

In [ ]:
Shp_Path = "/content/drive/MyDrive/Tree Species Project/Sampling Points/TreeSpecies60mPointInside.shp"
SmpPnts = gpd.read_file(Shp_Path)
print(f'{SmpPnts} Cloumns : {SmpPnts.columns}')
print(f'Shape of Sampling {SmpPnts.shape}')
SmpPnts[["Median", "Mean", "STD","MAX","MIN","Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9","Q10"]]=np.nan
print(f'New Shape of Sampling {SmpPnts.shape}')

                                    IMG_ID      ID BT_DEU          BT_ENG  \
0          Fagus_sylvatica_7_138225_BI_NLF  138225     Bu  European beech   
1          Fagus_sylvatica_7_138342_BI_NLF  138342     Bu  European beech   
2          Fagus_sylvatica_7_138464_BI_NLF  138464     Bu  European beech   
3              Picea_abies_4_138466_BI_NLF  138466     Fi          Spruce   
4      Acer_pseudoplatanus_7_138484_BI_NLF  138484    BAh  Sycamore maple   
...                                    ...     ...    ...             ...   
50376         Tilia_spec._9_99911_WEFL_NLF   99911     Li          Linden   
50377         Tilia_spec._3_99951_WEFL_NLF   99951     Li          Linden   
50378         Tilia_spec._4_99957_WEFL_NLF   99957     Li          Linden   
50379         Tilia_spec._4_99969_WEFL_NLF   99969     Li          Linden   
50380        Tilia_spec._4_100000_WEFL_NLF  100000     Li          Linden   

                    BT_BOT BT_GEN AKL    SOURCE      l0          l1  \
0   

In [ ]:
MySampling_Years=['2017','2018','2019','2020']
SmpPnts_Target= SmpPnts.query('YEAR in @MySampling_Years')
print(f'Shape of the Target Years {SmpPnts_Target.shape}')

Shape of the Target Years (22201, 31)


In [ ]:
canopyHeight = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').mosaic()
#region = ee.Geometry.BBox(-122.0859, 37.0436, -122.0626, 37.0586)

In [ ]:
c=1
for index, row in SmpPnts_Target[:].iterrows():
  print(index)
  print(SmpPnts_Target.shape)
  c=c+1
print(c)

In [ ]:
Step_Quntile=np.arange(0.1, 1.1, 0.1)
c=1
for index, row in SmpPnts_Target[11000:].iterrows():
    xx=row['geometry'].x
    yy=row['geometry'].y
    point=ee.Geometry.Point([xx,yy])
    aoi=point.buffer(5).bounds()
    url = canopyHeight.getDownloadUrl({'region': aoi,'dimensions':'10x10','format': 'NPY'})
    response = requests.get(url)
    data = np.load(io.BytesIO(response.content))
    #print(data)
    #print(data.shape)
    #print(data.dtype)
    # Convert Types
    data = data.astype(np.float64)
    #print(data)
    ## Compute
    Median_T=np.median(data)
    Mean_T = np.mean(data)
    STD_T = np.std(data)
    Max_T=np.max(data)
    Min_T=np.min(data)
    Q_T=np.quantile(data,Step_Quntile)
    #print("Mean:", Mean_T)
    #print("Median:", Median_T)
    #print("STD:", STD_T)
    #print("Max:", Max_T)
    #print("Min:", Min_T)
    #print("Quntile:", Q_T)
    #
    SmpPnts_Target.loc[index, ['Median', 'Mean',"STD","MAX","MIN","Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9","Q10"]] = [Median_T, Mean_T,STD_T,Max_T,Min_T,*Q_T]
    c=c+1
    print(f"Counter is {c}")
    print("######")
#print(SmpPnts_Target.loc[0])
SmpPnts_Target.to_file("/content/drive/MyDrive/Tree Species Project/Sampling Point Canopy/SamplingPntsCanopyP2.shp")
print("Finished")

NameError: name 'np' is not defined